## spark read s3 delta

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import Window, Row, SparkSession

import pprint
import boto3
import json
import sys
import os

pp = pprint.PrettyPrinter(indent = 3)
print('imported modules.')

ModuleNotFoundError: No module named 'boto3'

### read s3

In [2]:
client = boto3.client('s3')
bucket = "yelp-dataset-stevenhurwitt"
file = "raw/yelp_academic_dataset_business.json"

bucket_meta = client.list_objects(Bucket = 'yelp-dataset-stevenhurwitt')
print('files in s3 bucket:')
print('')
for c in bucket_meta['Contents']:
    print(c['Key'])

files in s3 bucket:

raw/
raw/yelp_academic_dataset_business.json
raw/yelp_academic_dataset_checkin.json
raw/yelp_academic_dataset_review.json
raw/yelp_academic_dataset_tip.json
raw/yelp_academic_dataset_user.json


In [3]:
def read_json(filename):
    """
    reads a yelp .json file from s3 bucket.

    keyword arguments:
    filename - name of file (str)

    returns: json_file (json)
    """

    bucket = "yelp-dataset-stevenhurwitt"
    file = "raw/yelp_academic_dataset_business.json"
    response = client.get_object(Bucket = bucket, Key = file)
    
    file_content = response['Body'].read().decode('utf-8')
    json_file = json.loads("[" + file_content.replace("}\n{", "},\n{") + "]")
    return(json_file)

## read json files

In [4]:
business_file = read_json("raw/yelp_academic_dataset_business.json")
checkin_file = read_json("raw/yelp_academic_dataset_checkin.json")
review_file = read_json("raw/yelp_academic_dataset_review.json")
tip_file = read_json("raw/yelp_academic_dataset_tip.json")
user_file = read_json("raw/yelp_academic_dataset_user.json")

print("read json files from s3.")

read json files from s3.


### review

In [5]:
pp.pprint(review_file[0])

{  'address': '1616 Chapala St, Ste 2',
   'attributes': {'ByAppointmentOnly': 'True'},
   'business_id': 'Pns2l4eNsfO8kk83dixA6A',
   'categories': 'Doctors, Traditional Chinese Medicine, '
                 'Naturopathic/Holistic, Acupuncture, Health & Medical, '
                 'Nutritionists',
   'city': 'Santa Barbara',
   'hours': None,
   'is_open': 0,
   'latitude': 34.4266787,
   'longitude': -119.7111968,
   'name': 'Abby Rappoport, LAC, CMQ',
   'postal_code': '93101',
   'review_count': 7,
   'stars': 5.0,
   'state': 'CA'}


### tip

In [6]:
pp.pprint(tip_file[0])

{  'address': '1616 Chapala St, Ste 2',
   'attributes': {'ByAppointmentOnly': 'True'},
   'business_id': 'Pns2l4eNsfO8kk83dixA6A',
   'categories': 'Doctors, Traditional Chinese Medicine, '
                 'Naturopathic/Holistic, Acupuncture, Health & Medical, '
                 'Nutritionists',
   'city': 'Santa Barbara',
   'hours': None,
   'is_open': 0,
   'latitude': 34.4266787,
   'longitude': -119.7111968,
   'name': 'Abby Rappoport, LAC, CMQ',
   'postal_code': '93101',
   'review_count': 7,
   'stars': 5.0,
   'state': 'CA'}


### user

In [7]:
pp.pprint(user_file[0])

{  'address': '1616 Chapala St, Ste 2',
   'attributes': {'ByAppointmentOnly': 'True'},
   'business_id': 'Pns2l4eNsfO8kk83dixA6A',
   'categories': 'Doctors, Traditional Chinese Medicine, '
                 'Naturopathic/Holistic, Acupuncture, Health & Medical, '
                 'Nutritionists',
   'city': 'Santa Barbara',
   'hours': None,
   'is_open': 0,
   'latitude': 34.4266787,
   'longitude': -119.7111968,
   'name': 'Abby Rappoport, LAC, CMQ',
   'postal_code': '93101',
   'review_count': 7,
   'stars': 5.0,
   'state': 'CA'}


### checkin

In [8]:
pp.pprint(checkin_file[0])

{  'address': '1616 Chapala St, Ste 2',
   'attributes': {'ByAppointmentOnly': 'True'},
   'business_id': 'Pns2l4eNsfO8kk83dixA6A',
   'categories': 'Doctors, Traditional Chinese Medicine, '
                 'Naturopathic/Holistic, Acupuncture, Health & Medical, '
                 'Nutritionists',
   'city': 'Santa Barbara',
   'hours': None,
   'is_open': 0,
   'latitude': 34.4266787,
   'longitude': -119.7111968,
   'name': 'Abby Rappoport, LAC, CMQ',
   'postal_code': '93101',
   'review_count': 7,
   'stars': 5.0,
   'state': 'CA'}


### business

In [9]:
pp.pprint(business_file[0])

{  'address': '1616 Chapala St, Ste 2',
   'attributes': {'ByAppointmentOnly': 'True'},
   'business_id': 'Pns2l4eNsfO8kk83dixA6A',
   'categories': 'Doctors, Traditional Chinese Medicine, '
                 'Naturopathic/Holistic, Acupuncture, Health & Medical, '
                 'Nutritionists',
   'city': 'Santa Barbara',
   'hours': None,
   'is_open': 0,
   'latitude': 34.4266787,
   'longitude': -119.7111968,
   'name': 'Abby Rappoport, LAC, CMQ',
   'postal_code': '93101',
   'review_count': 7,
   'stars': 5.0,
   'state': 'CA'}


In [10]:
dynamodb = boto3.resource('dynamodb', endpoint_url="https://us-east-2.console.aws.amazon.com?arn=arn:aws:dynamodb:us-east-2134132211607:8000")
print('created dynamo resource.')

# try:
#     yelp_business = dynamodb.create_table(
#             TableName='yelp.business',
#             KeySchema=[
#                 {
#                     'AttributeName': 'business_id',
#                     'KeyType': 'HASH'  # Partition key
#                 }
#             ],
#             AttributeDefinitions=[
#                 {
#                     'AttributeName': 'name',
#                     'AttributeType': 'S'
#                 }
#             ],
#             ProvisionedThroughput={
#                 'ReadCapacityUnits': 25,
#                 'WriteCapacityUnits': 20
#             }
#         )
#     print('created dynamo table.')

# except Exception as e:
#     print(f"exception: {e}")
#     print("failed to create dynamo table.")

created dynamo resource.


In [11]:
print(bucket) 

yelp-dataset-stevenhurwitt


In [12]:
print(file)

raw/yelp_academic_dataset_business.json


In [13]:
print(len(business_file))
print(len(checkin_file))
# review_file = read_json("raw/yelp_academic_dataset_review.json")
# tip_file = read_json("raw/yelp_academic_dataset_tip.json")
# user_file = read_json("raw/yelp_academic_dataset_user.json")


150346
150346


In [14]:
print('business json file has {} records with size of {} mb.'.format(len(business_file), sys.getsizeof(business_file)/1000000))
print('tip json file has {} records with size of {} mb.'.format(len(tip_file), sys.getsizeof(tip_file)/1000000))
print('user json file has {} records with size of {} mb.'.format(len(user_file), sys.getsizeof(user_file)/1000000))
print('review json file has {} records with size of {} mb.'.format(len(review_file), sys.getsizeof(review_file)/1000000))
print('checkin json file has {} records with size of {} mb.'.format(len(checkin_file), sys.getsizeof(checkin_file)/1000000))


business json file has 150346 records with size of 1.28316 mb.
tip json file has 150346 records with size of 1.28316 mb.
user json file has 150346 records with size of 1.28316 mb.
review json file has 150346 records with size of 1.28316 mb.
checkin json file has 150346 records with size of 1.28316 mb.


In [15]:
# df_pandas = business.toPandas()

In [16]:
# html_df = df_pandas.to_html()

In [17]:
# display(html_df)

In [18]:
# ! gcloud auth login --no-launch-browser